In [1]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

print(torch.cuda.is_available())

False


/home/robinferede/miniconda3/envs/quad/lib/python3.11/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /opt/conda/conda-bld/pytorch_1695392035891/work/c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


**Load Dataset**

In [2]:
# trajectories containing 199 points
dataset_path = 'datasets/2D_QUAD_HOVER_EPS=0.5.npz'

print('loading dataset...')
dataset = np.load(dataset_path)
dataset = dict(dataset)
num = len(dataset['y'])
length = len(dataset['y'][0])
print(num, 'trajectories of length', length)

loading dataset...
100000 trajectories of length 79


**View Dataset**

In [3]:
# view 10 random trajectories
indices = np.random.choice(num, 1)
trajectories = [{key: dataset[key][i] for key in dataset.keys()} for i in indices]

from quadcopter_animation import animation2D
animation2D.animate(*trajectories)

**Create Dataloader**

In [4]:
# the network should learn the mapping from
# X = [y, z, vy, vz, theta, omega] to U = [ul, ur]

# now we make an dataloader that loads an (X,U) pair
class MyDataset(Dataset):
    def __init__(self, trajectory_indices):
        y       = dataset['y'][trajectory_indices].flatten()
        z       = dataset['z'][trajectory_indices].flatten()
        vy      = dataset['vy'][trajectory_indices].flatten()
        vz      = dataset['vz'][trajectory_indices].flatten()
        theta   = dataset['theta'][trajectory_indices].flatten()
        omega   = dataset['omega'][trajectory_indices].flatten()
        self.X  = torch.tensor(np.stack([y, z, vy, vz, theta, omega], axis=1), dtype=torch.float32)
        
        ul      = dataset['ul'][trajectory_indices].flatten()
        ur      = dataset['ur'][trajectory_indices].flatten()
        self.U  = torch.tensor(np.stack([ul, ur], axis=1), dtype=torch.float32)
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.U[idx]
    
# train/test split
batchsize_train = 256
batchsize_test = 4096
train_trajectories = range(int(0.8*num))
test_trajectories = list(set(range(num)) - set(train_trajectories))

train_dataset = MyDataset(train_trajectories)
train_loader = DataLoader(train_dataset, batch_size=batchsize_train, shuffle=True)

test_dataset = MyDataset(test_trajectories)
test_loader = DataLoader(test_dataset, batch_size=batchsize_test, shuffle=False)

**Define Neural Net**

In [7]:
from torch import nn

model = nn.Sequential(
    nn.Linear(train_dataset.X.shape[1], 120),
    nn.ReLU(),
    nn.Linear(120, 120),
    nn.ReLU(),
    nn.Linear(120, 120),
    nn.ReLU(),
    nn.Linear(120, train_dataset.U.shape[1]),
    nn.Sigmoid()
)

print(model)

Sequential(
  (0): Linear(in_features=6, out_features=120, bias=True)
  (1): ReLU()
  (2): Linear(in_features=120, out_features=120, bias=True)
  (3): ReLU()
  (4): Linear(in_features=120, out_features=120, bias=True)
  (5): ReLU()
  (6): Linear(in_features=120, out_features=2, bias=True)
  (7): Sigmoid()
)


**Define Loss Function and Optimizer**

In [8]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False) 

**Training Loop**

In [ ]:
 from tqdm import tqdm

# loop over the dataset multiple times
num_epochs = 3
for epoch in range(num_epochs):
    loop = tqdm(enumerate(train_loader), total=len(train_loader), leave=False)
    for i, (data, targets) in loop:
        # zero the parameter gradients
        optimizer.zero_grad()
        
        # forward + backward + optimize
        outputs = model(data)
        
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        # update progressbar
        loop.set_description(f"Epoch [{epoch+1}/{num_epochs}]")
        loop.set_postfix(loss=loss.item())
    loop.close()
    # save model
    torch.save(model, 'neural_networks/2D_QUAD_HOVER_EPS=0.5.pt')
print('Finished Training')

**Simulate**

In [9]:
torch.load('neural_networks/2D_QUAD_HOVER_EPS=0.5.pt')

Sequential(
  (0): Linear(in_features=6, out_features=120, bias=True)
  (1): ReLU()
  (2): Linear(in_features=120, out_features=120, bias=True)
  (3): ReLU()
  (4): Linear(in_features=120, out_features=120, bias=True)
  (5): ReLU()
  (6): Linear(in_features=120, out_features=2, bias=True)
  (7): Sigmoid()
)

In [10]:
# controller
def controller(x):
    with torch.no_grad():
        u = model(torch.tensor(x, dtype=torch.float32))
    return u.numpy()


# dynamics:
def f(x,u):
    # state
    y,z,vy,vz,theta,omega = x
    
    # control
    ul,ur = u
    
    # parameters
    g = 9.81
    m = 0.389
    Ixx = 0.001242
    L = 0.08
    maxthrust = 2.35
    minthrust = 1.76
    M = (maxthrust - minthrust)
    F = 2*minthrust
    beta = 0.5
    
    # dynamics
    dy_dt       = vy
    dz_dt       = vz
    dv_ydt      = -((ur+ul) / m * M + F / m) * np.sin(theta) - beta * vy
    dvz_dt      = ((ur+ul) / m * M + F / m) * np.cos(theta) - g - beta * vz
    dtheta_dt   = omega
    domega_dt   = L / Ixx * M * (ur - ul)
    
    return np.array([dy_dt, dz_dt, dv_ydt, dvz_dt, dtheta_dt, domega_dt])
    
# simulate
def simulate(y0,z0,vy0,vz0,theta0,omega0,T):
    dt = 0.01
    t = np.linspace(0, T, int(T/dt))
    
    x = np.array([y0,z0,vy0,vz0,theta0,omega0])
    X = [x]
    U = []
    for i in range(len(t)-1):
        u = controller(x)
        x = x + f(x,u)*dt
        X.append(x)
        U.append(u)
    u = controller(x)
    U.append(u)
    
    # outputs
    X = np.array(X)
    U = np.array(U)
    return {
        't':        t,
        'y':        X[:,0],
        'z':        X[:,1],
        'vy':       X[:,2],
        'vz':       X[:,3],
        'theta':    X[:,4],
        'omega':    X[:,5],
        'ul':       U[:,0],
        'ur':       U[:,1]
    }

In [11]:
from quadcopter_animation import animation2D
import importlib
importlib.reload(animation2D)

# simulate from initial condition
traj = simulate(y0=5, z0=5, vy0=0, vz0=0, theta0=0, omega0=0, T=10)
animation2D.animate(traj)

In [103]:
# simulate multiple
num = 100
y0 = np.random.uniform(-10.0, 10.0, num)
z0 = np.random.uniform(-10.0, 10.0, num)
vy0 = np.random.uniform(-5.,5., num)
vz0 = np.random.uniform(-5.,5., num)
theta0 = np.random.uniform(-np.pi/3, np.pi/3, num)
omega0 = np.random.uniform(-0.01, -0.01, num)

trajectories = [simulate(y0=y0[i], z0=z0[i], vy0=vy0[i], vz0=vz0[i], theta0=theta0[i], omega0=omega0[i], T=10) for i in range(num)]
animation2D.animate(*trajectories)

**Compare simulated trajectory with dataset**

In [104]:
# trajectory from dataset
index = 0
traj_dataset = {key: dataset[key][index] for key in dataset.keys()}

# simulated trajectory with the same initial condition
initial_condition = [traj_dataset[key][0] for key in ['y','z','vy','vz','theta','omega']]
T = traj_dataset['t'][-1]
traj_simulated = simulate(*initial_condition, T)

# animate side by side
animation2D.animate(traj_dataset, traj_simulated)